In [ ]:
import mysql.connector
import pandas as pd
import os

# Paths to all CSVs
csv_paths = {
    year: f"C:/Users/Akshiya George/OneDrive/Desktop/Data Science/Amazon/amazon_india_{year}.csv"
    for year in range(2015, 2026)
}

# Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Akshiya13",
    database="amazon",
    allow_local_infile=True
)
cursor = conn.cursor()

# Enable local infile
cursor.execute("SET GLOBAL local_infile = 1;")

# Function to save temp CSV and load into MySQL
def load_table(df, table_name):
    temp_path = f"{table_name}_temp.csv"
    df.to_csv(temp_path, index=False)
    query = f"""
    LOAD DATA LOCAL INFILE '{temp_path.replace("\\", "/")}'
    INTO TABLE {table_name}
    FIELDS TERMINATED BY ',' 
    ENCLOSED BY '"'
    LINES TERMINATED BY '\n'
    IGNORE 1 ROWS;
    """
    cursor.execute(query)
    conn.commit()
    os.remove(temp_path)
    print(f"✅ Loaded {table_name}")

# Process each file
for year, path in csv_paths.items():
    df = pd.read_csv(path)

    # Extract and deduplicate
    transactions = df[[
        'transaction_id', 'order_date', 'customer_id', 'product_id',
        'original_price_inr', 'discount_percent', 'discounted_price_inr',
        'quantity', 'subtotal_inr', 'delivery_charges', 'final_amount_inr',
        'payment_method', 'payment_method_cleaned', 'delivery_days',
        'delivery_type', 'is_prime_member', 'is_festival_sale', 'festival_name',
        'customer_rating', 'return_status', 'order_month', 'order_year',
        'order_quarter', 'dup_key', 'dup_count', 'dup_status', 'flag_for_review'
    ]].drop_duplicates()

    products = df[[
        'product_id', 'product_name', 'category', 'subcategory', 'brand',
        'product_weight_kg', 'is_prime_eligible', 'product_rating'
    ]].drop_duplicates()

    customers = df[[
        'customer_id', 'customer_city', 'customer_state',
        'customer_tier', 'customer_spending_tier', 'customer_age_group'
    ]].drop_duplicates()

    time_dimension = df[['order_date']].drop_duplicates().copy()
    time_dimension['day'] = pd.to_datetime(time_dimension['order_date']).dt.day
    time_dimension['month'] = pd.to_datetime(time_dimension['order_date']).dt.month
    time_dimension['year'] = pd.to_datetime(time_dimension['order_date']).dt.year
    time_dimension['quarter'] = pd.to_datetime(time_dimension['order_date']).dt.quarter
    time_dimension['week'] = pd.to_datetime(time_dimension['order_date']).dt.isocalendar().week
    time_dimension['day_of_week'] = pd.to_datetime(time_dimension['order_date']).dt.day_name()
    time_dimension['is_weekend'] = time_dimension['day_of_week'].isin(['Saturday', 'Sunday'])
    time_dimension['is_holiday'] = df.groupby('order_date')['is_festival_sale'].max().values
    time_dimension['holiday_name'] = df.groupby('order_date')['festival_name'].first().values
    time_dimension.rename(columns={'order_date': 'date'}, inplace=True)

    # Load into MySQL
    load_table(customers, "customers")
    load_table(products, "products")
    load_table(time_dimension, "time_dimension")
    load_table(transactions, "transactions")

# Create indexes
index_queries = [
    # Transactions
    "CREATE INDEX idx_order_date ON transactions(order_date);",
    "CREATE INDEX idx_customer_id ON transactions(customer_id);",
    "CREATE INDEX idx_product_id ON transactions(product_id);",
    "CREATE INDEX idx_payment_method ON transactions(payment_method_cleaned);",
    "CREATE INDEX idx_order_year_month ON transactions(order_year, order_month);",
    "CREATE INDEX idx_return_status ON transactions(return_status);",
    # Products
    "CREATE INDEX idx_category_subcategory ON products(category, subcategory);",
    "CREATE INDEX idx_brand ON products(brand);",
    "CREATE INDEX idx_prime_eligible ON products(is_prime_eligible);",
    # Customers
    "CREATE INDEX idx_customer_location ON customers(customer_city, customer_state);",
    "CREATE INDEX idx_customer_tier ON customers(customer_tier);",
    "CREATE INDEX idx_spending_tier ON customers(customer_spending_tier);",
    # Time Dimension
    "CREATE INDEX idx_year_month ON time_dimension(year, month);",
    "CREATE INDEX idx_day_of_week ON time_dimension(day_of_week);",
    "CREATE INDEX idx_is_weekend ON time_dimension(is_weekend);",
    "CREATE INDEX idx_is_holiday ON time_dimension(is_holiday);"
]

for query in index_queries:
    try:
        cursor.execute(query)
        print(f"🔧 Index created: {query.split()[2]}")
    except mysql.connector.Error as err:
        print(f"⚠️ Index creation failed: {err}")

cursor.close()
conn.close()
print("🎉 All data loaded and indexed successfully.")

In [ ]:
import streamlit as st
import pandas as pd
from sqlalchemy import create_engine

# --- Configuration ---
st.set_page_config(page_title="Database Viewer", layout="wide")

# --- Secrets (stored in .streamlit/secrets.toml) ---
db_user = st.secrets["amazon"]["root"]
db_password = st.secrets["amazon"]["Akshiya"]
db_host = st.secrets["amazon"]["localhost"]
db_port = st.secrets["amazon"]["3306"]
db_name = st.secrets["amazon"]["amazon"]

# --- Create Engine ---
engine = create_engine(f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

# --- Load Data ---
@st.cache_data
def load_data():
    query = "SELECT * FROM your_table_name"
    return pd.read_sql(query, engine)

# --- Streamlit UI ---
st.title("📊 MySQL Database Viewer")
df = load_data()
st.dataframe(df, use_container_width=True)

In [ ]:
# %%
import mysql.connector
import pandas as pd
import os
import streamlit as st
from sqlalchemy import create_engine

# --- Streamlit Page Config ---
st.set_page_config(page_title="📊 Amazon MySQL Viewer", layout="wide")

# --- Load Secrets ---
db_user = st.secrets["amazon"]["root"]
db_password = st.secrets["amazon"]["Akshiya"]
db_host = st.secrets["amazon"]["localhost"]
db_port = st.secrets["amazon"]["3306"]
db_name = st.secrets["amazon"]["amazon"]

# --- Create SQLAlchemy Engine ---
engine = create_engine(f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

# --- Connect to MySQL for Bulk Loading ---
conn = mysql.connector.connect(
    host=db_host,
    user=db_user,
    password=db_password,
    database=db_name,
    allow_local_infile=True
)
cursor = conn.cursor()
cursor.execute("SET GLOBAL local_infile = 1;")

# --- CSV Paths ---
csv_paths = {
    year: f"C:/Users/Akshiya George/OneDrive/Desktop/Data Science/Amazon/amazon_india_{year}.csv"
    for year in range(2015, 2026)
}

# --- Load Table Function ---
def load_table(df, table_name):
    temp_path = f"{table_name}_temp.csv"
    df.to_csv(temp_path, index=False)
    query = f"""
    LOAD DATA LOCAL INFILE '{temp_path.replace("\\", "/")}'
    INTO TABLE {table_name}
    FIELDS TERMINATED BY ',' 
    ENCLOSED BY '"'
    LINES TERMINATED BY '\n'
    IGNORE 1 ROWS;
    """
    cursor.execute(query)
    conn.commit()
    os.remove(temp_path)
    print(f"✅ Loaded {table_name}")

# --- Process Each CSV ---
for year, path in csv_paths.items():
    df = pd.read_csv(path)

    transactions = df[[
        'transaction_id', 'order_date', 'customer_id', 'product_id',
        'original_price_inr', 'discount_percent', 'discounted_price_inr',
        'quantity', 'subtotal_inr', 'delivery_charges', 'final_amount_inr',
        'payment_method', 'payment_method_cleaned', 'delivery_days',
        'delivery_type', 'is_prime_member', 'is_festival_sale', 'festival_name',
        'customer_rating', 'return_status', 'order_month', 'order_year',
        'order_quarter', 'dup_key', 'dup_count', 'dup_status', 'flag_for_review'
    ]].drop_duplicates()

    products = df[[
        'product_id', 'product_name', 'category', 'subcategory', 'brand',
        'product_weight_kg', 'is_prime_eligible', 'product_rating'
    ]].drop_duplicates()

    customers = df[[
        'customer_id', 'customer_city', 'customer_state',
        'customer_tier', 'customer_spending_tier', 'customer_age_group'
    ]].drop_duplicates()

    time_dimension = df[['order_date']].drop_duplicates().copy()
    time_dimension['day'] = pd.to_datetime(time_dimension['order_date']).dt.day
    time_dimension['month'] = pd.to_datetime(time_dimension['order_date']).dt.month
    time_dimension['year'] = pd.to_datetime(time_dimension['order_date']).dt.year
    time_dimension['quarter'] = pd.to_datetime(time_dimension['order_date']).dt.quarter
    time_dimension['week'] = pd.to_datetime(time_dimension['order_date']).dt.isocalendar().week
    time_dimension['day_of_week'] = pd.to_datetime(time_dimension['order_date']).dt.day_name()
    time_dimension['is_weekend'] = time_dimension['day_of_week'].isin(['Saturday', 'Sunday'])
    time_dimension['is_holiday'] = df.groupby('order_date')['is_festival_sale'].max().values
    time_dimension['holiday_name'] = df.groupby('order_date')['festival_name'].first().values
    time_dimension.rename(columns={'order_date': 'date'}, inplace=True)

    load_table(customers, "customers")
    load_table(products, "products")
    load_table(time_dimension, "time_dimension")
    load_table(transactions, "transactions")

# --- Create Indexes ---
index_queries = [
    "CREATE INDEX idx_order_date ON transactions(order_date);",
    "CREATE INDEX idx_customer_id ON transactions(customer_id);",
    "CREATE INDEX idx_product_id ON transactions(product_id);",
    "CREATE INDEX idx_payment_method ON transactions(payment_method_cleaned);",
    "CREATE INDEX idx_order_year_month ON transactions(order_year, order_month);",
    "CREATE INDEX idx_return_status ON transactions(return_status);",
    "CREATE INDEX idx_category_subcategory ON products(category, subcategory);",
    "CREATE INDEX idx_brand ON products(brand);",
    "CREATE INDEX idx_prime_eligible ON products(is_prime_eligible);",
    "CREATE INDEX idx_customer_location ON customers(customer_city, customer_state);",
    "CREATE INDEX idx_customer_tier ON customers(customer_tier);",
    "CREATE INDEX idx_spending_tier ON customers(customer_spending_tier);",
    "CREATE INDEX idx_year_month ON time_dimension(year, month);",
    "CREATE INDEX idx_day_of_week ON time_dimension(day_of_week);",
    "CREATE INDEX idx_is_weekend ON time_dimension(is_weekend);",
    "CREATE INDEX idx_is_holiday ON time_dimension(is_holiday);"
]

for query in index_queries:
    try:
        cursor.execute(query)
        print(f"🔧 Index created: {query.split()[2]}")
    except mysql.connector.Error as err:
        print(f"⚠️ Index creation failed: {err}")

cursor.close()
conn.close()
print("🎉 All data loaded and indexed successfully.")

# --- Streamlit UI ---
@st.cache_data
def load_data():
    query = "SELECT * FROM transactions"  # Replace with your desired table
    return pd.read_sql(query, engine)

st.title("📊 Amazon MySQL Database Viewer")
df = load_data()
st.dataframe(df, use_container_width=True)